In [31]:
import pandas as pd
from selenium import webdriver
import chromedriver_binary#sin necesidad de añadir a Path
import time

In [32]:
browser = webdriver.Chrome()#manejador
browser.get("https://www.worldbank.org/en/projects-operations/procurement/debarred-firms#")

time.sleep(15)#espera para buscar

rows_num = len(browser.find_elements_by_xpath('//*[@id="k-debarred-firms"]/div[3]/table/tbody/tr'))
print(rows_num)#numero total de datos

2083


In [33]:
if rows_num == 0:
    print("Uff parece que la pagina no esta disponible actualmente")
    print("Intenta de nuevo mas tarde. Suele tardar un momento en actualizarse")

In [34]:
#====================== Obtencion de datos ============================================
#------- Arrays para almacenar
dic_firm_name =[]
dic_adress=[]
dic_country=[]
dic_date_from = []
dic_date_to= []
dic_grounds=[]

In [35]:
#------------ ciclos para extraer datos de la tabla
i=1
u=1
while i <= rows_num:
    #-------- Xpatch para extraer datos
    xname = '//*[@id="k-debarred-firms"]/div[3]/table/tbody/tr[{}]/td[1]'.format(i)
    xdirec = '//*[@id="k-debarred-firms"]/div[3]/table/tbody/tr[{}]/td[3]'.format(i)
    xcountry = '//*[@id="k-debarred-firms"]/div[3]/table/tbody/tr[{}]/td[4]'.format(i)
    xfromdate = '//*[@id="k-debarred-firms"]/div[3]/table/tbody/tr[{}]/td[5]'.format(i)
    xtodate = '//*[@id="k-debarred-firms"]/div[3]/table/tbody/tr[{}]/td[6]'.format(i)
    xgrounds = '//*[@id="k-debarred-firms"]/div[3]/table/tbody/tr[{}]/td[7]'.format(i)
    
    #obtengo los valores deseados y agrego a los arrays
    name = browser.find_element_by_xpath(xname).text
    direc = browser.find_element_by_xpath(xdirec).text
    contry = browser.find_element_by_xpath(xcountry).text
    fromdate = browser.find_element_by_xpath(xfromdate).text
    todate = browser.find_element_by_xpath(xtodate).text
    grounds = browser.find_element_by_xpath(xgrounds).text
    
    #---llenando arrays
    dic_firm_name.append(name)
    dic_adress.append(direc)
    dic_country.append(contry)
    dic_date_from.append(fromdate)
    dic_date_to.append(todate)
    dic_grounds.append(grounds)
    
    if (i%100 == 0):
        print("Van {}".format(i))
        print(dic_firm_name[i-1])

    i+=1
     

Van 100
CHINA RAILWAY FIRST GROUP FORTH ENGINEERING CO., LTD.(Reg. No.: 916104007197681415)*503
Van 200
EMMYWAYS ENGINEERING/CONSTRUCTION LIMITED*487
Van 300
SHANGHAI SHUXIANG REAL ESTATE CO., LTD.(上海蜀翔房地产有限公司)(Reg. No: 91310114MA1GTWLF51)*482
Van 400
QIQIHAR FUTIE RAIL SLEEPER CO., LTD.(齐齐哈尔富铁轨枕有限公司)(Reg. No: 912302061285163568)*482
Van 500
REAL ESTATE INVESTMENT CO., LTD. OF CHINA RAILWAY 16TH BUREAU GROUP CO.,LTD.(中铁十六局集团置业投资有限公司)(Reg. No: 91110000576914526P)*482
Van 600
REAL ESTATE DEVELOPMENT CO., LTD OF CHINA RAILWAY 22ND BUREAU GROUP CO., LTD(中铁二十二局集团房地产开发有限公司)(Reg. No: 91110000569497701Q)*482
Van 700
CHINA RAILWAY CONSTRUCTION PRECAST COMPONENT R&D CONSULTING (SHANGHAI) CO., LTD.(中铁建预制构件研发咨询（上海）有限公司)(Reg. No: 91310117MA1J21T044)*482
Van 800
CHINA RAILWAY 18TH BUREAU GROUP MADAGASCAR CO., LTD.(中铁十八局集团马达加斯加公司)(Reg. No: 2008-001124)*482
Van 900
CCECC (MALAWI) CO., LTD.(中国土木工程集团（马拉维）有限公司)(Reg. No: N1000201600256)*482
Van 1000
BHARUCH ENVIRO INFRASTRUCTURE LTD.(Reg. No.: U45300GJ199

In [37]:
datos = {'FIRM NAME':dic_firm_name, 
         'ADDRESS':dic_adress, 
         'COUNTRY':dic_country,
         'INELIGIBITY_PERIOD_FROM':dic_date_from,
         'INELIGIBITY_PERIOD_TO':dic_date_to,
         'GROUNDS':dic_grounds}
dfwb = pd.DataFrame(datos)

## Limpieza

In [39]:
dfwb.head()

,FIRM NAME,ADDRESS,COUNTRY,INELIGIBITY_PERIOD_FROM,INELIGIBITY_PERIOD_TO,GROUNDS
0,LLC “WOODMANMEBEL”*514,"YUROVSKAYA STREET 92, OFFICE NO. 1, MOSCOW 125466",Russian Federation,31-Dec-2019,30-Dec-2022,"Procurement Guidelines, 1.16(a)(ii)"
1,ZENG ZILING,"3-21-01, E-SHIDAI, XIN MAO YUAN, SI GONG LI, C...",China,24-Dec-2019,11-Sep-2023,Cross Debarment: ADB
2,GENERAL ELECTRIC POWER SWEDEN AB,"SLOTTSGATAN 69, SE-602 22 NORRKOPING",Sweden,17-Dec-2019,26-Nov-2025,Cross Debarment: EBRD
3,ESWARI ELECTRICALS PVT. LIMITED*513,"PLOT NO. 64, INDUSTRIAL ESTATE, PERUNGUDI, CHE...",India,26-Nov-2019,25-Nov-2023,"Procurement Guidelines, 1.16(a)(ii)"
4,MOTO-ID S.A.(Reg. No.: B171868)*475,"6, RUE EUGENE RUPPERT, L-2453",Luxembourg,22-Nov-2019,23-Oct-2025,"May 2010 Procurement Guidelines, Paras. 1.14(a..."


In [41]:
#reemplazar caracteres que dan conflicto al insertar en la tabla
dfwb['FIRM NAME'] = dfwb['FIRM NAME'].str.replace('“','')
dfwb['FIRM NAME'] = dfwb['FIRM NAME'].str.replace('”','')
dfwb['FIRM NAME'] = dfwb['FIRM NAME'].str.replace("'",' ')
dfwb['ADDRESS'] = dfwb['ADDRESS'].str.replace("'",' ')
dfwb['COUNTRY'] = dfwb['COUNTRY'].str.replace("'",' ')
#se dejan los nan 

## Guardando en la BD

In [68]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.types import String
import io
con = psycopg2.connect(database="prueba", user="postgres", password="12345", host="127.0.0.1", port="5432")

In [69]:
cur = con.cursor()

In [70]:
cur.execute('''CREATE TABLE IF NOT EXISTS diccionario
(
    "firmname" TEXT,
    "adress" TEXT,
    "country" TEXT,
    "ineligibilityperiodfromdate" TEXT,
    "ineligibilityperiodtodate" TEXT,
    "grounds" TEXT
);''')
#    "additionalfirminfo" TEXT,
print("Tabla creada")
con.commit()
con.close()

Tabla creada


In [71]:
con = psycopg2.connect(database="prueba", user="postgres", password="12345", host="127.0.0.1", port="5432")
cur= con.cursor()
i=0
num= dfwb.shape[0]
while i <= num-1:
    consulta = "INSERT INTO diccionario(firmname,adress,country,ineligibilityperiodfromdate,ineligibilityperiodtodate,grounds) VALUES ('{}', '{}', '{}', '{}', '{}', '{}')".format(
        dfwb['FIRM NAME'][i], dfwb['ADDRESS'][i],
        dfwb['COUNTRY'][i], dfwb['INELIGIBITY_PERIOD_FROM'][i],dfwb['INELIGIBITY_PERIOD_TO'][i], dfwb['GROUNDS'][i])
    cur.execute(consulta);
    con.commit()
    i+=1
    
con.close()